1. Importación de librerías

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    classification_report
)

print("Librerías importadas correctamente.")



Librerías importadas correctamente.


2. Cargar dataset procesado

In [2]:
def find_project_root(start_path):
    current = start_path
    while True:
        if os.path.exists(os.path.join(current, ".git")):
            return current
        parent = os.path.dirname(current)
        if parent == current:
            raise RuntimeError("No se encontró la raíz del proyecto (.git).")
        current = parent

PROJECT_ROOT = find_project_root(os.getcwd())
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
DATASET_PATH = os.path.join(DATA_PROCESSED, "dataset_modelado.parquet")
MODEL_PATH = os.path.join(PROJECT_ROOT, "models","sprint4")
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATASET_PATH:", DATASET_PATH)


PROJECT_ROOT: c:\IA_Investigacion\DetCorr_Limpio
DATASET_PATH: c:\IA_Investigacion\DetCorr_Limpio\data\processed\dataset_modelado.parquet


In [3]:
df = pd.read_parquet(DATASET_PATH)

print("Dimensiones:", df.shape)
df.head()


Dimensiones: (14179, 42)


,CODIGO_UNICO,SECTOR,DEPARTAMENTO,NIVEL_GOBIERNO,PROCESO,OBJETO_PROCESO,CODIGO_OBRA,METODO_CONTRATACION,TIEMPO_ABSOLUCION_CONSULTAS,TIEMPO_PRESENTACION_OFERTAS,...,MES,PLANIFICADO,REAL,IND_INTERVENSION,IND_RESIDENTE,IND_MONTO_ADELANTO_MATERIALES,IND_MONTO_ADELANTO_DIRECTO,IND_FECHA_ADELANTO_MATERIALES,IND_FECHA_ADELANTO_DIRECTO,riesgo
0,2002060.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,3.0,CONSULTORÍA DE OBRA,19777.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
1,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,60.0,CONSULTORÍA DE OBRA,826.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
2,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,60.0,CONSULTORÍA DE OBRA,826.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
3,2015918.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,24.0,CONSULTORÍA DE OBRA,45660.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
4,2026767.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,14.0,CONSULTORÍA DE OBRA,143536.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0


3. Prep. columnas

In [4]:
assert "riesgo" in df.columns, "No existe la columna 'riesgo' en el dataset."

y = df["riesgo"]
X = df.drop(columns=["riesgo"])

print("Columnas X:", X.shape[1])
print("Valores objetivo:", y.unique())



Columnas X: 41
Valores objetivo: [0. 1.]


4. Detectar columnas categóricas y numéricas

In [5]:
categoricas = X.select_dtypes(include=["object"]).columns.tolist()
numericas = X.select_dtypes(exclude=["object"]).columns.tolist()

print("Categóricas:", categoricas)
print("Numéricas:", numericas)


Categóricas: ['CODIGO_UNICO', 'SECTOR', 'DEPARTAMENTO', 'NIVEL_GOBIERNO', 'OBJETO_PROCESO', 'METODO_CONTRATACION', 'ESTADO_OBRA', 'ETAPA', 'IND_INTERVENSION', 'IND_RESIDENTE', 'IND_MONTO_ADELANTO_MATERIALES', 'IND_MONTO_ADELANTO_DIRECTO']
Numéricas: ['PROCESO', 'CODIGO_OBRA', 'TIEMPO_ABSOLUCION_CONSULTAS', 'TIEMPO_PRESENTACION_OFERTAS', 'CODIGO_RUC', 'CONVOCATORIA_PROCESO_GANADO', 'TOTALPROCESOSPARTICIPANTES', 'CODIGO_CONTRATO', 'MONTO_CONTRACTUAL', 'MONTO_REFERENCIAL', 'MONTO_OFERTADO_PROMEDIO', 'CONVOCATORIA', 'DNI_MIEMBRO_COMITE', 'CODIGO_RUC_GANADOR', 'CODIGO_RUC_PARTICIPANTE', 'RUC_GANADOR', 'RUC_PARTICIPANTE', 'MONTO_OFERTADO', 'DIAS_PLAZO', 'TOTAL_CONTROL_PREVIO', 'TOTAL_CONTROL_SIMULTANEO', 'TOTAL_CONTROL_POSTERIOR', 'RIESGO_OBRA', 'ANHO', 'MES', 'PLANIFICADO', 'REAL', 'IND_FECHA_ADELANTO_MATERIALES', 'IND_FECHA_ADELANTO_DIRECTO']


5. Preprocesamiento con ColumnTransformer

In [6]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categoricas),
        ("num", "passthrough", numericas)
    ]
)

print("Preprocesador configurado.")


Preprocesador configurado.


6. Definir modelos con pipeline

In [7]:
models = {
    "RandomForest": Pipeline([
        ("prep", preprocess),
        ("clf", RandomForestClassifier(random_state=42))
    ]),
    "XGBoost": Pipeline([
        ("prep", preprocess),
        ("clf", XGBClassifier(
            eval_metric="logloss",
            tree_method="hist",
            random_state=42
        ))
    ]),
    "LogReg": Pipeline([
        ("prep", preprocess),
        ("clf", LogisticRegression(
            max_iter=2000
        ))
    ])    
}

print("Modelos configurados.")



Modelos configurados.


7. Grids refinados (solo parámetros del clf)

In [8]:
param_grids = {
    "RandomForest": {
        "clf__n_estimators": [200, 400],
        "clf__max_depth": [10, 20, None],
        "clf__min_samples_split": [2, 5]
    },
    "XGBoost": {
        "clf__n_estimators": [200, 400],
        "clf__max_depth": [4, 6],
        "clf__learning_rate": [0.05, 0.1],
        "clf__subsample": [0.8, 1.0]
    },
    "LogReg": {
        "clf__C": [0.1, 1.0, 10],
        "clf__penalty": ["l2"],
        "clf__solver": ["lbfgs", "liblinear"]
    }    
}

print("Grids listos.")


Grids listos.


8. Train/Test Split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Split hecho.")
print(X_train.shape, X_test.shape)


Split hecho.
(11343, 41) (2836, 41)


9. Entrenamiento con GridSearchCV

In [10]:
best_models = {}
results = {}

for name, model in models.items():
    print(f"\n=== Optimizando {name} ===")
    
    grid = GridSearchCV(
        model,
        param_grids[name],
        scoring="f1",
        cv=3,
        n_jobs=-1,
        error_score="raise"
    )
    
    grid.fit(X_train, y_train)
    
    best_models[name] = grid.best_estimator_
    
    y_pred = grid.predict(X_test)
    y_proba = grid.predict_proba(X_test)[:, 1]
    
    results[name] = {
        "best_params": grid.best_params_,
        "f1": f1_score(y_test, y_pred),
        "roc_auc": roc_auc_score(y_test, y_proba)
    }

results



=== Optimizando RandomForest ===

=== Optimizando XGBoost ===

=== Optimizando LogReg ===


{'RandomForest': {'best_params': {'clf__max_depth': None,
   'clf__min_samples_split': 2,
   'clf__n_estimators': 200},
  'f1': 0.24130879345603273,
  'roc_auc': 0.4930260167464115},
 'XGBoost': {'best_params': {'clf__learning_rate': 0.1,
   'clf__max_depth': 6,
   'clf__n_estimators': 400,
   'clf__subsample': 0.8},
  'f1': 0.09823182711198428,
  'roc_auc': 0.49747906698564587},
 'LogReg': {'best_params': {'clf__C': 0.1,
   'clf__penalty': 'l2',
   'clf__solver': 'lbfgs'},
  'f1': 0.0,
  'roc_auc': 0.5029865430622009}}

10. Seleccionar el mejor modelo

In [11]:
df_results = pd.DataFrame(results).T
df_results_sorted = df_results.sort_values(by="f1", ascending=False)

print("Resultados ordenados:")
df_results_sorted



Resultados ordenados:


,best_params,f1,roc_auc
RandomForest,"{'clf__max_depth': None, 'clf__min_samples_spl...",0.241309,0.493026
XGBoost,"{'clf__learning_rate': 0.1, 'clf__max_depth': ...",0.098232,0.497479
LogReg,"{'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__so...",0.0,0.502987


In [12]:
import os
import pandas as pd

# -----------------------------
# Guardar resultados de modelos
# -----------------------------
df_resultados = pd.DataFrame.from_dict(results, orient="index")
df_resultados.reset_index(inplace=True)
df_resultados.rename(columns={"index": "modelo"}, inplace=True)

# Ruta del archivo
output_csv = os.path.join(MODEL_PATH, "resultados_modelos.csv")

# Crear carpeta si no existe
os.makedirs(MODEL_PATH, exist_ok=True)

# Guardar CSV
df_resultados.to_csv(output_csv, index=False)

print("Archivo generado:", output_csv)
df_resultados


Archivo generado: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4\resultados_modelos.csv


,modelo,best_params,f1,roc_auc
0,RandomForest,"{'clf__max_depth': None, 'clf__min_samples_spl...",0.241309,0.493026
1,XGBoost,"{'clf__learning_rate': 0.1, 'clf__max_depth': ...",0.098232,0.497479
2,LogReg,"{'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__so...",0.000000,0.502987


11. Guardar modelo ganador

In [13]:
import joblib
MODELS_DIR = os.path.join(PROJECT_ROOT, "models", "sprint4")
os.makedirs(MODELS_DIR, exist_ok=True)

best_model_name = df_results_sorted.index[0]
best_model = best_models[best_model_name]

output_path = os.path.join(MODELS_DIR, f"modelo_final_{best_model_name}.pkl")
joblib.dump(best_model, output_path)

print("Modelo guardado en:", output_path)


Modelo guardado en: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4\modelo_final_RandomForest.pkl


In [14]:
best_model_name, results[best_model_name]


('RandomForest',
 {'best_params': {'clf__max_depth': None,
   'clf__min_samples_split': 2,
   'clf__n_estimators': 200},
  'f1': 0.24130879345603273,
  'roc_auc': 0.4930260167464115})